# Bi-LSTM model on PUBHEALTH data set 

The first model would be only for the title column with a binary target Fake/Not Fake.
This  Bi-LSTM model was built on the first LSTM Model (that was done by Andrii Shchur in 2020. Here is its [article](https://towardsdatascience.com/fake-news-detector-with-deep-learning-approach-part-i-eda-757f5c052)

- The Pubhealth dataset contains helath's topics 
- It contains these attributes : 
  - claim_id: unique id for a news article
  - claim
  - date_published
  - explanation
  - fact_checkers
  - main_text
  - sources
  - label (TRUE, FALSE, mixture, unproven and snopes)
  - subjects

<img src='https://media.giphy.com/media/fTsXis2dANPustUVfG/giphy.gif'>

### <span style="background-color: #FFFF00">In this notebook, I have corrected the preprocessing of the data</span>

## Importing packages 

In [1]:
# Load libraries
import pandas as pd
import numpy as np
import os.path as op
import time
import matplotlib.pyplot as plt
#Loading keras 
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import one_hot
#Loading nltk
import nltk
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
#Loading sklearn
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, f1_score, roc_auc_score
#Loading K-folds
from sklearn.model_selection import StratifiedKFold
#Loading the model 
import sys  
sys.path.insert(0, '../../models')
from PY.model4 import get_model

Using TensorFlow backend.


## Paths 

In [2]:
DATA_DIR = '../../../../Data'
CODE_DIR_H5='../../models/H5'
# Create a path for the saving location of the model
MODEL_DIR_H5 = CODE_DIR_H5 + '/model4_PubHealth.h5'

## Loading Data 

In [3]:
# Load train data
train = pd.read_csv(op.join(DATA_DIR, 'PUBHEALTH/Claims_to_work_with/claims.csv'))
train 

title  \
0      Bat from Shawnee County tests positive for rab...   
1      Germany has banned pork from school canteens b...   
2      Coronavirus prompts Canada to roll out safe dr...   
3      Wayne National Forest plans fires for tree, wi...   
4      School closures will have little impact on COV...   
...                                                  ...   
11869  Christmas miracle for Florida dog whose heart ...   
11870  Another dolphin dies at Arizona aquatic facility.   
11871  What the facts say" is ... "the best scenario ...   
11872  FDA advisory panel rejects J&J drug for acute ...   
11873  Gene-editing Chinese scientist kept much of hi...   

                                                    text  \
0      Topeka television station KSNT reports that th...   
1      On 7 March 2016, British tabloid Express repor...   
2      In March, the Canadian government urged provin...   
3      Forest officials say scientists who study nati...   
4      Data on the effects of school closures on COVI...   
...                                                  ...   
11869  The heart of Gerald Ford’s 1-year-old French b...   
11870  A statement from Dolphinaris Arizona in Scotts...   
11871  Republican National Committee chairman Reince ...   
11872  The panel voted 10 to 0, with one abstention, ...   
11873  He Jiankui’s outsized aspirations began to tak...   

                                                  author  label  
0      https://www.ksnt.com/news/bat-tests-positive-f...      0  
1      http://bnp.org.uk/news/regional/bnp-victory-br...      1  
2                                                             0  
3                                                             0  
4                                                             0  
...                                                  ...    ...  
11869  https://www.gainesville.com/news/20200102/holi...      0  
11870                                                         0  
11871  https://www.nytimes.com/2016/07/13/us/politics...      1  
11872                                                         0  
11873  https://twitter.com/larsonchristina,http://bit...      0  

[11874 rows x 4 columns]

In [6]:
# Find Na 
train.isnull().sum()

title    0
text     0
label    0
dtype: int64

In [7]:
#Get the Depndent feature
X_train=train.drop('label',axis=1)
y_train=train['label']

In [8]:
# set vocabulary size
vo_size=500
messages=X_train.copy()
messages.reset_index(inplace=True)

In [9]:
len(messages), len(y_train)

(10075, 10075)

In [10]:
# Dataset Preprocessing
cmp=0
ps =PorterStemmer()
corpus = []
for i in range(0, len(messages)):
    print("Status: %s / %s" %(i, len(messages)), end="\r")
    review = re.sub('[^a-zA-Z]', ' ',messages['title'][i])
    review = review.lower()
    review = review.split()
    
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    if review :
        review = ' '.join(review)
        corpus.append(review)
        y_train[i]=y_train[i]
    else :
        cmp=cmp+1
        y_train[i]=np.nan
y_train.reset_index()

Status: 15 / 10075

C:\Users\CBS Compter\anaconda3\envs\tensorflow7\lib\site-packages\ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app


index  label
0          0      0
1          1      1
2          2      0
3          3      0
4          4      0
...      ...    ...
10070  10070      0
10071  10071      0
10072  10072      1
10073  10073      0
10074  10074      0

[10075 rows x 2 columns]

In [11]:
print('the number of lines that could not be preprocessed',cmp)

the number of lines that could not be preprocessed 0


In [12]:
print("We need to delete those",y_train.isnull().sum(),"lines from the label column")
y_train=y_train.dropna()
y_train.astype('Int64')

We need to delete those 0 lines from the label column


0        0
1        1
2        0
3        0
4        0
        ..
10070    0
10071    0
10072    1
10073    0
10074    0
Name: label, Length: 10075, dtype: Int64

In [13]:
# one hot representation
onehot_rep = [one_hot(words, vo_size) for words in corpus]

In [14]:
# pad_sequences
sent_length = 20
embedded_doc=pad_sequences(onehot_rep, padding='pre', maxlen=sent_length)
print(embedded_doc)

[[  0   0   0 ... 329 385 205]
 [  0   0   0 ... 304  71   5]
 [  0   0   0 ... 302 152 375]
 ...
 [  0   0   0 ... 463 198 178]
 [  0   0   0 ...  64 324 372]
 [  0   0   0 ... 498 123 301]]


In [15]:
embedding_vector_feature = 10

In [16]:
# check shape 
len(embedded_doc),y_train.shape

(10075, (10075,))

In [17]:
# final data for NN
X_final=np.array(embedded_doc)
y_final=np.array(y_train)
X_final.shape,y_final.shape

((10075, 20), (10075,))

## Training  the model 

In [ ]:
start=time.time()
# fix random seed for reproducibility
seed = 7
np.random.seed(seed)
# define 10-fold cross validation test harness
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=seed)
cvscores1 = []
cvscores2 = []
cvscores3 = []
cvscores4 = []
num_iter=0

for train, test in kfold.split(X_final, y_final):
    num_iter=num_iter+1
    model=get_model(vo_size,embedding_vector_feature,sent_length)
    # Fit the model
    history=model.fit(X_final[train], y_final[train], epochs=10, batch_size=64,verbose=0)
    # evaluate the model
    scores = model.evaluate(X_final[test], y_final[test], verbose=0)
    print("------subset number", num_iter,"------")
    print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))
    cvscores1.append(scores[1] * 100)
    print("%s: %.2f%%" % (model.metrics_names[2], scores[2]*100))
    cvscores2.append(scores[2] * 100)
    print("%s: %.2f%%" % (model.metrics_names[3], scores[3]*100))
    cvscores3.append(scores[3] * 100)
    print("%s: %.2f%%" % (model.metrics_names[4], scores[4]*100))
    cvscores4.append(scores[4] * 100)
print("--------------------------------------------" )   
print("accuracy","%.2f%% (+/- %.2f%%)" % (np.mean(cvscores1), np.std(cvscores1)))
print("precision","%.2f%% (+/- %.2f%%)" % (np.mean(cvscores2), np.std(cvscores2)))
print("recall","%.2f%% (+/- %.2f%%)" % (np.mean(cvscores3), np.std(cvscores3)))
print("f1-mesure","%.2f%% (+/- %.2f%%)" % (np.mean(cvscores4), np.std(cvscores4)))
end=time.time()

------subset number 1 ------
accuracy: 72.32%
precision_mesure: 69.26%
recall_mesure: 47.27%
f1_mesure: 54.96%
------subset number 2 ------
accuracy: 70.34%
precision_mesure: 61.31%
recall_mesure: 55.11%
f1_mesure: 57.42%
------subset number 3 ------
accuracy: 70.83%
precision_mesure: 62.95%
recall_mesure: 52.68%
f1_mesure: 55.91%
------subset number 4 ------
accuracy: 69.84%
precision_mesure: 60.20%
recall_mesure: 51.11%
f1_mesure: 54.25%
------subset number 5 ------
accuracy: 72.12%
precision_mesure: 64.41%
recall_mesure: 59.31%
f1_mesure: 60.52%
------subset number 6 ------
accuracy: 70.41%
precision_mesure: 61.62%
recall_mesure: 58.95%
f1_mesure: 59.33%


In [ ]:
# Save the model
model.save(MODEL_DIR_H5)
model.summary() 

In [ ]:
print('time of training' , end-start)

## Evaluation metrics

In [ ]:
# summarize history for accuracy
plt.plot(history.history['accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.show()

In [ ]:
# summarize history for loss
plt.plot(history.history['loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.show()

In [ ]:
# summarize history for f1-mesure
plt.plot(history.history['f1_mesure'])
plt.title('model F1-mesure')
plt.ylabel('F1-mesure')
plt.xlabel('epoch')
plt.show()

In [ ]:
# summarize history for precision
plt.plot(history.history['precision_mesure'])
plt.title('model precision')
plt.ylabel('precision')
plt.xlabel('epoch')
plt.show()

In [ ]:
# summarize history for recall
plt.plot(history.history['recall_mesure'])
plt.title('model recall')
plt.ylabel('recall')
plt.xlabel('epoch')
plt.show()